In [102]:

import transformers
from datetime import datetime
import os
import random
import json
pid = os.getpid()
seed = int(pid)+int(datetime.now().timestamp())
print("seed: ", seed)
random.seed(seed)

out_dir = "data"
os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")



seed:  1727109561


In [1]:
from vllm import LLM, SamplingParams

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
#model_id="meta-llama/Meta-Llama-3.1-8B-Instruct"

tensor_parallel_size = 8
# トークナイザーとモデルの準備
model = LLM(
    model=model_id,
    trust_remote_code=True,
    max_model_len=2000,
    tensor_parallel_size=tensor_parallel_size,
)

INFO 09-17 17:31:41 config.py:904] Defaulting to use mp for distributed inference
INFO 09-17 17:31:41 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llam

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


(VllmWorkerProcess pid=543385) INFO 09-17 17:31:53 weight_utils.py:242] Using model weights format ['*.safetensors']
INFO 09-17 17:32:00 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543391) INFO 09-17 17:32:00 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543389) INFO 09-17 17:32:01 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543390) INFO 09-17 17:32:01 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543388) INFO 09-17 17:32:01 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543386) INFO 09-17 17:32:02 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543385) INFO 09-17 17:32:03 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=543387) INFO 09-17 17:32:03 model_runner.py:1008] Loading model weights took 16.4605 GB
INFO 09-17 17:32:07 distri

In [2]:
temperature=0.7
top_k=50

In [13]:
def llm_gen(model,prompt_list,temperature=0.7,top_k=50):
    outputs = model.generate(
        prompt_list,
        sampling_params=SamplingParams(
            temperature=temperature,
            max_tokens=1024,
            #repetition_penalty=1.2,
            top_k=top_k,
        )
    )
    return [i.outputs[0].text.strip() for i in outputs]

In [ ]:

tokenizer=transformers.AutoTokenizer.from_pretrained(model_id)

In [24]:
from datasets import load_dataset

ds=load_dataset("kanhatakeyama/material-properties",split="train")
ds=ds.shuffle()

{'CompName': 'unknown',
 'SMILES': 'C1=CC(=CC(=C1)CN)CN',
 'Property': 'Boiling temperature',
 'Value': 247.2222222,
 'unit': '[oC]',
 'Source': 'Wikipedia/Wikidata'}

In [45]:


def gen_problem(record):
    comp_name=record["CompName"]
    smiles=record["SMILES"]
    unit=record["unit"]
    property_name=record["Property"]
    q=f"""Predict the {property_name} {unit} of the following compound. 
    #Restriction: Output must contain "#Reason" section which explains the step-by-step quantitative reasoning behind the prediction. Output must also contain "#Predicted" section which contains only the predicted value (number only).
    #Name: {comp_name}
    #SMILES: {smiles}"""
    actual_value=record["Value"]

    chat = [
        {"role": "user", "content": q},
    ]


    prompt=tokenizer.apply_chat_template(chat,tokenize=False,)
    assist_prefix="assistant\n\n#Reason\n"
    prompt+=assist_prefix

    return prompt,actual_value

In [107]:
from tqdm import tqdm
n_records=100
for _ in tqdm(range(100)):
    problems=[]
    prompt_list=[
    ]
    for i in range(n_records):
        record=random.choice(ds)
        prompt,actual_value=gen_problem(record)
        problems.append(
            {
            "record":record,
            "prompt":prompt,
            "actual_value":actual_value   
            }

        )
        prompt_list.append(prompt)

    #interference
    predicted_text_list=llm_gen(model,prompt_list)


    #parse
    good_records=[]
    for i in range(n_records):
        problems[i]["CompoundName"]=problems[i]["record"]["CompName"]
        problems[i]["SMILES"]=problems[i]["record"]["SMILES"]
        problems[i]["Property"]=problems[i]["record"]["Property"]
        problems[i]["Unit"]=problems[i]["record"]["unit"]
        problems[i]["predicted"]=predicted_text_list[i].split("#Predicted\n")[-1]
        problems[i]["predicted_text"]=predicted_text_list[i]
        #誤差率
        problems[i]["actual_value"]=float(problems[i]["actual_value"])
        try:
            error=abs((problems[i]["actual_value"]-float(problems[i]["predicted"]))/problems[i]["actual_value"])
            problems[i]["error_rate"]=error
        except:
            continue

        with open(f"data/{current_time_no_symbols}_llm_gen.jsonl","a", encoding='utf-8') as f:
            f.write(json.dumps(problems[i],ensure_ascii=False)+"\n")

  0%|          | 0/100 [00:00<?, ?it/s]

 34%|███▍      | 34/100 [13:58<27:06, 24.65s/it]


KeyboardInterrupt: 

In [98]:
import pandas as pd
#df=pd.DataFrame(problems)
#df=df.drop(columns=["record","prompt"])
#df.to_csv("predicted.csv",index=False)

In [99]:
#df

,actual_value,CompoundName,SMILES,Property,Unit,predicted,predicted_text,error_rate
0,52.000000,Dicobalt octacarbonyl,O=C=[Co]1(=C=O)(=C=O)C(=O)[Co](=C=O)(=C=O)(=C=...,Boiling temperature,[oC],152,To predict the boiling temperature of dicobalt...,1.923077
1,-1.000000,Benzoyl chloride,ClC(=O)c1ccccc1,Melting temperature,[oC],39.5,To predict the melting temperature of Benzoyl ...,40.500000
2,7.500000,unknown,CC1=CC=C(C=C1)N,Ionization Energy,[eV],6.5,To predict the ionization energy of the given ...,0.133333
3,1.100000,unknown,CC(=CC1=CC=CC=C1)[N+](=O)[O-],Density,[g/cm3],1.34,"To predict the density of the given compound, ...",0.218182
4,1.900000,unknown,C(=O)(C(=O)O)O,Density,[g/cm3],1.05,"To predict the density of the given compound, ...",0.447368
5,166.000000,unknown,C1=CC=C(C=C1)[Ga](C2=CC=CC=C2)C3=CC=CC=C3,Melting temperature,[oC],0,To predict the melting temperature of the give...,1.000000
6,10.000000,unknown,CCC(C)OC(=O)C,Vapor pressure,[kPa],0.018,To predict the vapor pressure of the given com...,0.998200
7,243.000000,unknown,C(C(F)(F)F)(Cl)Br,Vapor pressure,[kPa],0.035,To predict the vapor pressure of the compound ...,0.999856
8,21.111111,unknown,CC(=CC(=O)OC)OP(=O)(OC)OC,Melting temperature,[oC],128.23,To predict the melting temperature of the give...,5.074053
9,15.300000,Ethylene,C=C,Absolute molar magnetic susceptibility,[10^-6 cm3/mol],0,To predict the absolute molar magnetic suscept...,1.000000
